# 데이터 타입, 함수, 연산자

In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 숫자형과 함수

- 빅쿼리는 숫자, 문자, 시간, 지리 데이터, 정형, 반정형 데이터등 다양한 데이터 타입을 지원
    - int64
        : 유일한 정수형 실수이면 float64, 불리언이면 bool
    - numeric
        : 정확한 계산이 필요할 때 적합
    - string
        : 가변 길이의 유니코드 문자열 표현
    - timestamp
        : 시간의 절대 시점
    - datetime
        : 달력상의 날짜와 시간
    - geography
        : 지구 표면의 점, 선, 폴리곤
    - struct, array
        : 구조체와 배열

query = '''
with example as(
    select 'sat' as day, 1451 as numrides, 1018 as oneways
    union all select 'sun',2376,936
)
select *,(oneways / numrides) as frac_oneway
from example
'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

- 함수유형
    - 스칼라(scalar) 함수 : 하나 이상의 입력 파라미터에서 작동해 단일 값을 반환하는 함수
        - ex) round( ), substr( )
    - 집계(aggregate) : 계산을 수행하고 단일 값을 반환하는 함수 , 종종 group by와 함께 사용  
        - ex) max( ), sum( ), count( ), avg( )
    - 분석(analytics)함수 : 값 모음에서 작동하지만 모음의 각 값에 대한 결과를 반환
        - ex) row_number( ), rank( )
    - 테이블 반환 함수 : from절에서 사용할 수 있는 결과 집합을 반환
        - ex) 배열에서 unnest를 호출한 다음 선택
    - 사용자 정의 함수 : 사용자가 직접 구현한 함수


## 수학 함수

In [3]:
# round함수를 사용해 결과를 반올림하는 쿼리
query = '''
with example as(
    select 'sat' as day, 1451 as numrides, 1018 as oneways
    union all select 'sun',2376,936
)
select *,round((oneways / numrides),2) as frac_oneway
from example
'''
df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.09rows/s]


,day,numrides,oneways,frac_oneway
0,sat,1451,1018,0.70
1,sun,2376,936,0.39


## 표준 규격 부동소수점 분할

In [5]:
# IEEE_DIVIDE를 사용한 쿼리
query = '''
with example as(
    select 'sat' as day, 1451 as numrides, 1018 as oneways
    union all select 'sun',2376,936
    union all select 'wnd',0,0
)

select *, round(ieee_divide(oneways,numrides),2) as frac_oneway
from example
'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.98rows/s]


,day,numrides,oneways,frac_oneway
0,sat,1451,1018,0.70
1,sun,2376,936,0.39
2,wnd,0,0,NaN


## safe 함수

In [6]:
query ='''
SELECT LOG(10,3), SAFE.lOG(10,-3)
'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.03rows/s]


,f0_,f1_
0,2.095903,NaN


## numeric을 사용한 정밀 소수 계산

In [15]:
#numeric을 사용한 소수점 계산
query = '''
with example as(
    select numeric '1.23' as payment
    union all select numeric'7.89'
    union all select numeric'12.43'
)

select
    sum(payment) as total_paid,
    avg(payment) as average_paid
from example 
'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df
# 평균은 반복되는 소수점이므로 numeric 타입으로도 정확한 계산이 불가능

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.92rows/s]


,total_paid,average_paid
0,21.55,7.183333333


# 불(bool) 다루기

## 논리연산

In [17]:
query='''
with example as (
    select null as is_vowel, null as letter, -1 as position
    union all select true, 'a',1
    union all select false, 'b',2
    union all select false, 'c',3
)

select *
from example 
where is_vowel != false'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.98rows/s]


,is_vowel,letter,position
0,True,a,1


In [18]:
query='''
with example as (
    select null as is_vowel, null as letter, -1 as position
    union all select true, 'a',1
    union all select false, 'b',2
    union all select false, 'c',3
)

select *
from example 
where is_vowel is not false'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

#비교연산자는 null과 비교하면 null을 반환하지않지만 is 연산자는 null값 또한 반환

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.93rows/s]


,is_vowel,letter,position
0,None,None,-1
1,True,a,1


## 조건식

In [19]:
query = '''
with catalog as(
    select 30.0 as costprice, 0.15 as markup, 0.1 as taxrate
    union all select null, 0.21, 0.15
    union all select 30.0, null, 0.09
    union all select 30.0, 0.30, null 
    union all select 30.03, null, null
)

select *, round( costprice * if(markup is null, 1.05, 1+markup)* if(taxrate is null, 1.10,1+taxrate),2) as salesprice
from catalog 
'''

df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df


Downloading: 100%|█████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 14.45rows/s]


,costprice,markup,taxrate,salesprice
0,30.00,0.15,0.10,37.95
1,NaN,0.21,0.15,NaN
2,30.00,NaN,0.09,34.34
3,30.00,0.30,NaN,42.90
4,30.03,NaN,NaN,34.68


## coalesce로 null 값 처리

In [5]:
query ='''
with catalog as(
    select 30.0 as costPrice, 0.15 as markup, 0.1 as taxRate
    union all select null, 0.21, 0.15
    union all select 30.0, null, 0.09
    union all select 30.0, 0.30, null
    union all select 30.0, null, null
)
select 
    *,round(coalesce(
    costPrice * (1+markup) * (1+taxrate),
    costPrice * 1.05 * (1+taxrate),
    costPrice * (1+markup) *1.10,
    null
    ),2) as salesPrice
    
from catalog
'''
df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 12.58rows/s]


,costPrice,markup,taxRate,salesPrice
0,30.0,0.15,0.10,37.95
1,NaN,0.21,0.15,NaN
2,30.0,NaN,0.09,34.34
3,30.0,0.30,NaN,42.90
4,30.0,NaN,NaN,NaN


## 타입 변환과 타입 강제

In [4]:
query = '''
with example as(
    select 'john' as employee, 'paternity leave' as hours_worked
    union all select 'janki', '35'
    union all select 'jian','vacation'
    union all select 'jose','40'
)
select sum(safe_cast(hours_worked as int64)) 
from example
'''
df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.76rows/s]


,f0_
0,75


In [5]:
query = '''
with example as(
    select 'john' as employee, '0' as hours_worked
    union all select 'janki', '35'
    union all select 'jian','0'
    union all select 'jose','40'
)
select sum(safe_cast(hours_worked as int64)) 
from example
'''
df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.67rows/s]


,f0_
0,75


## 불리언 변환을 피하기 위해 COUNTIF 사용하기

In [7]:
query='''
with example as(
    select true as is_vowel, 'a' as letter, 1 as position
    union all select false, 'b',2
    union all select false, 'c',3
)
select countif(is_vowel) as num_vowel
from example
'''
df = pd.read_gbq(query=query, dialect='standard',project_id = 'project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.89rows/s]


,num_vowel
0,1


# 문자열 함수

In [11]:
query ='''
with example as(
    select *
    from unnest([
        'SEOUL','SUWON','INCHEON']) as city
)
select city, length(city)as len, lower(city)as lower,strpos(city,'ON') as orpos
from example
'''
# strpos(a,b) : a 내부에 b 위치를 정수값으로 반환
df= pd.read_gbq(query = query, dialect='standard',project_id='project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.87rows/s]


,city,len,lower,orpos
0,SEOUL,5,seoul,0
1,SUWON,5,suwon,4
2,INCHEON,7,incheon,6


In [8]:
# substr, concat를 사용한 쿼리

query = '''with example as(
    select 'armin@abc.com' as email, 'annapolis, md' as city
    union all select 'boyan@bca.com', 'boulder, co'
    union all select 'carrie@cab.com','chicago, il'
)
select
    concat( 
        substr(email,1, strpos(email,'@') -1), ' from ',city) as callers 
from example
'''
# SUBSTR(value, position, length]) : value값 내부에서 position에서부터 length만큼 데이터 반환
df= pd.read_gbq(query = query, dialect='standard',project_id='project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  6.74rows/s]


,callers
0,"armin from annapolis, md"
1,"boyan from boulder, co"
2,"carrie from chicago, il"


## 출력 및 파싱

In [12]:
query = '''
select 
cast(42 as string), 
cast('42'as int64),
format('%03d', 42),
format('%5.3f',32.457842),
format('%5.3f',32.4),
format('***%s***','H'),
format('%s-%03d','Agent',7)
'''
df= pd.read_gbq(query = query, dialect='standard',project_id='project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.84rows/s]


,f0_,f1_,f2_,f3_,f4_,f5_,f6_
0,42,42,042,32.458,32.400,***H***,Agent-007


## 변환 함수

In [13]:
query='''
select
    LPAD('hello',10,'*'),
    rpad('hello',10,'*'),
    ltrim(' hello '),
    rtrim(' hello '),
    trim(' hello'),
    trim('***hello***', '*'),
    reverse('hello')
'''
df= pd.read_gbq(query = query, dialect='standard',project_id='project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.00rows/s]


,f0_,f1_,f2_,f3_,f4_,f5_,f6_
0,*****hello,hello*****,hello,hello,hello,hello,olleh


## 정규표현식

In [7]:
query = '''
select
    column,
    regexp_contains(column, r'\d{5}(?:[-\s]\d{4})?') has_zipcode,
    regexp_contains(column, r'^\d{5}(?:[-\s]\d{4})?$') is_zipcode,
    regexp_extract(column, r'\d{5}(?:[-\s]\d{4})?') the_zipcode,
    regexp_extract_all(column,r'^\d{5}(?:[-\s]\d{4})?') the_zipcode2,
    regexp_replace(column, r'\d{5}(?:[-\s]\d{4})?','******') masked
from (
    select*
    from unnest(['12345','1234','12345-9876',
    'abc 12345 def', 'abcde-fghi',
    '12345 ab 34567', '1234 9876'
    ])as column
)
'''
df= pd.read_gbq(query = query, dialect='standard',project_id='project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 21.00rows/s]


,column,has_zipcode,is_zipcode,the_zipcode,the_zipcode2,masked
0,12345,True,True,12345,[12345],******
1,1234,False,False,None,[],1234
2,12345-9876,True,True,12345-9876,[12345-9876],******
3,abc 12345 def,True,False,12345,[],abc ****** def
4,abcde-fghi,False,False,None,[],abcde-fghi
5,12345 ab 34567,True,False,12345,[12345],****** ab ******
6,1234 9876,False,False,None,[],1234 9876


## 타임 스탬프

In [16]:
query = '''
select t1,t2, timestamp_diff(t1,t2, hour)
from (select 
    timestamp "2021-07-16 16:00:00.45" as t1,
    timestamp "2021-07-15 16:15:00.45+1" as t2
)
'''
df= pd.read_gbq(query = query, dialect='standard',project_id='project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.00rows/s]


,t1,t2,f0_
0,2021-07-16 16:00:00.450000+00:00,2021-07-15 15:15:00.450000+00:00,24


In [20]:
# 문자열 파싱
query = '''
select 
    fmt, input, zone, parse_timestamp(fmt, input, zone) as output
from (
    select '%Y%m%d-%H%M%S' as fmt, '20181118-164300' as input, '+0' as zone
    union all select '%c', 'Sat Nov 24 21:26:00 2018', 'America/Los_Angeles' 
    union all select '%x %X', '11/18/18 22:08:00','UTC'
)
'''
# %c : 날짜 및 시간 표현
# %x : 날짜를 mm/dd
df= pd.read_gbq(query = query, dialect='standard',project_id='project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.86rows/s]


,fmt,input,zone,output
0,%Y%m%d-%H%M%S,20181118-164300,+0,2018-11-18 16:43:00+00:00
1,%c,Sat Nov 24 21:26:00 2018,America/Los_Angeles,2018-11-25 05:26:00+00:00
2,%x %X,11/18/18 22:08:00,UTC,2018-11-18 22:08:00+00:00


In [21]:
query = '''
select
    ts, fmt, format_timestamp(fmt, ts, '+6') as ts_output
from(
    select current_timestamp() as ts, '%Y%m%d-%H%M%S' AS fmt
    union all select current_timestamp() as ts, '%c' as fmt
    union all select current_timestamp() as ts, '%x %X' as fmt
)
'''
# %c : 날짜 및 시간 표현
# %x : 날짜를 mm/dd/yy 형태로 표현
# %X : 시간을  hh:mm:ss

df= pd.read_gbq(query = query, dialect='standard',project_id='project-bigquery-319110',auth_local_webserver=True)
df

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  8.56rows/s]


,ts,fmt,ts_output
0,2021-07-17 09:16:46.645380+00:00,%Y%m%d-%H%M%S,20210717-151646
1,2021-07-17 09:16:46.645380+00:00,%c,Sat Jul 17 15:16:46 2021
2,2021-07-17 09:16:46.645380+00:00,%x %X,07/17/21 15:16:46
